# Обработка аудио данных

In [2]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import pandas as pd
import numpy as np
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
folder_path = '/content/drive/MyDrive/СРОД/data/audio/'

audio_paths = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
audio_paths

['Я тебя захуярю.mp3',
 'Я тоже хочу шоколадку.mp3',
 'Повар спрашивает повара.mp3',
 'Стетхем мотивация.mp3',
 'Ну дa ну да пошел я.mp3',
 'Быков о как.mp3']

In [9]:
display(Audio(folder_path+"Ну дa ну да пошел я.mp3", autoplay=True))

In [15]:
data = pd.read_csv(folder_path+'audio.csv')
data

,audio_path,emo
0,Быков о как.mp3,удивление
1,Ну дa ну да пошел я.mp3,сарказм
2,Повар спрашивает повара.mp3,радость
3,Стетхем мотивация.mp3,мотивация
4,Я тебя захуярю.mp3,гнев
5,Я тоже хочу шоколадку.mp3,грусть


In [16]:
def compute_mfcc_stats(file_path):
    y, sr = librosa.load(file_path, sr=None)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)

    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)

    return np.concatenate([mfcc_mean, mfcc_std])

In [18]:
mfcc_features = []
for index, row in data.iterrows():
    file_path = row['audio_path']
    mfcc_stats = compute_mfcc_stats(folder_path+file_path)
    mfcc_features.append(mfcc_stats)

In [19]:
mfcc_columns = [f'mfcc_mean_{i}' for i in range(1, 21)] + [f'mfcc_std_{i}' for i in range(1, 21)]
data_mfcc = pd.DataFrame(mfcc_features, columns=mfcc_columns)

In [22]:
data_final = pd.concat([data, data_mfcc], axis=1)
data_final

,audio_path,emo,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,...,mfcc_std_11,mfcc_std_12,mfcc_std_13,mfcc_std_14,mfcc_std_15,mfcc_std_16,mfcc_std_17,mfcc_std_18,mfcc_std_19,mfcc_std_20
0,Быков о как.mp3,удивление,-306.217377,155.866440,-57.177074,3.340175,-22.068521,-11.898156,-5.213050,-5.099892,...,8.522659,7.926084,7.375909,7.694225,7.148672,8.529389,8.628377,9.215812,6.921108,7.727834
1,Ну дa ну да пошел я.mp3,сарказм,-469.817108,172.359375,-24.958874,18.179810,-2.455525,17.733454,-0.720584,9.433302,...,9.125415,9.234423,12.692337,6.027408,6.590552,5.407841,6.706001,5.762805,5.240680,5.778743
2,Повар спрашивает повара.mp3,радость,-285.281555,162.419571,-26.404177,11.021117,2.777080,-12.947932,6.881361,-7.459101,...,13.245124,13.356621,12.818851,12.492974,12.292986,12.338556,11.551041,10.364025,9.472366,8.564305
3,Стетхем мотивация.mp3,мотивация,-262.667603,162.973663,16.401300,17.336012,2.119050,12.435620,5.952801,3.582277,...,10.122613,8.322286,9.066026,10.098476,9.603258,8.887160,7.625645,10.869247,7.930254,7.703794
4,Я тебя захуярю.mp3,гнев,-351.808044,178.149780,17.823978,6.425814,4.569681,13.475801,8.940997,4.917269,...,10.042687,8.492947,8.326096,9.184773,8.244258,6.920201,7.401228,6.266017,6.362381,6.809525
5,Я тоже хочу шоколадку.mp3,грусть,-352.745941,80.860435,-1.824689,27.126980,-17.457319,1.611422,4.099113,4.541307,...,17.620390,16.645359,14.381089,16.928652,16.586460,20.312141,19.201290,17.210220,14.520745,14.094327


In [23]:
data_final.to_csv(folder_path+'data_final.csv', encoding='utf-8')